In [76]:
import pandas as pd
import numpy as np
from scipy.stats import norm
import vartools as vt

In [77]:
def var_stocks(data, positions, nc, long):
    rend = data.pct_change().dropna()
    port_value = (data.iloc[-1] * positions).sum()
    w = (data*positions).iloc[-1] / port_value
    port_rend = np.dot(w, rend.T)
    
    if long == True:
        var_p = np.percentile(port_rend, 100-nc)
        var_m = (data * positions).sum(axis=1).iloc[-1] * (var_p)
        es_p = port_rend[port_rend < var_p].mean()
        es_m = (data * positions).sum(axis=1).iloc[-1] * (es_p)
    else:
        var_p = np.percentile(port_rend, nc)
        var_m = (data * positions).sum(axis=1).iloc[-1] * (var_p)
        es_p = port_rend[port_rend > var_p].mean()
        es_m = (data * positions).sum(axis=1).iloc[-1] * (es_p)
        
    resultados = pd.DataFrame({
        '':['%', '$'],
        'VaR':[var_p, var_m],
        'C-VaR':[es_p, es_m]
        })
    return resultados


def var_es(data, positions, nc, long):
    rend_inversion = ((data * positions).sum(axis=1)).pct_change().dropna()
    if long == True:
        percentil =  (1-nc)*100
        var_p = np.percentile(rend_inversion, percentil)
        var_m = (data * positions).sum(axis=1).iloc[-1] * (var_p)
        es_p = rend_inversion[rend_inversion < var_p].mean()
        es_m = (data * positions).sum(axis=1).iloc[-1] * (es_p)
    else:
        percentil = nc*100
        var_p = np.percentile(rend_inversion, percentil)
        var_m = (data * positions).sum(axis=1).iloc[-1] * (var_p)
        es_p = rend_inversion[rend_inversion > var_p].mean()
        es_m = (data * positions).sum(axis=1).iloc[-1] * (es_p)
    resultados = pd.DataFrame({'VAR %':var_p, 'VAR $':var_m, 'ES %':es_p, 'ES $':es_m}, index=['Resultados'])
    return resultados

call_blsdelta = lambda s, k, r, sigma, T: norm.cdf( ( np.log(s/k) + (r + sigma**2 / 2) * T ) / ( sigma * np.sqrt(T)) )
put_blsdelta = lambda s, k, r, sigma, T: np.abs(norm.cdf( ( np.log(s/k) + (r + sigma**2 / 2) * T ) / ( sigma * np.sqrt(T)) ) - 1)

def delta_hedge(calls, puts, noc_c, noc_p):
    return np.array([call_blsdelta(*i) for i in calls]).dot(noc_c) - np.array([put_blsdelta(*i) for i in puts]).dot(noc_p)
    

In [78]:
df = pd.read_excel('repasoexcel.xlsx')[['Date', 'QQQ', 'VOO', 'VXX', 'ITOT']]
df['Date'] = pd.to_datetime(df['Date'])
df = df.set_index('Date')
df.head()

,QQQ,VOO,VXX,ITOT
Date,,,,
2023-01-03,263.3283,345.8708,55.9600,83.4379
2023-01-04,264.5828,348.3611,54.4400,84.1397
2023-01-05,260.4409,344.5564,54.8800,83.1414
2023-01-06,267.6295,352.3931,53.6000,85.0391
2023-01-09,269.3619,352.1361,53.8800,85.0490


In [79]:
positions = [3967, 1576, 1023, 2032]
nc= 99
long = False

results = var_stocks(df, positions, nc, long)
results

,,VaR,C-VaR
0,%,0.0239,0.0259
1,$,"58,450.6747","63,322.6158"


In [80]:
vt.var_stocks(df, positions, 99, False, df.columns.tolist())

,Métrica,Porcentaje,cash
0,VaR,0.0239,"58,450.6747"
1,cVaR,0.0259,"63,322.6158"


In [81]:
df.pct_change().dropna().corr()

,QQQ,VOO,VXX,ITOT
QQQ,1.0000,0.9175,-0.6241,0.9020
VOO,0.9175,1.0000,-0.7325,0.9945
VXX,-0.6241,-0.7325,1.0000,-0.7395
ITOT,0.9020,0.9945,-0.7395,1.0000


In [82]:
df2 = pd.read_excel('repasoexcel.xlsx', sheet_name='Data_FX')
df2['Date'] = pd.to_datetime(df2['Date'])
df2 = df2.set_index('Date')
df2.head()

,CHFMXN=X,JPYMXN=X,USDMXN=X
Date,,,
2023-01-02,21.0175,0.1487,19.4466
2023-01-03,21.0539,0.1488,19.4588
2023-01-04,20.7346,0.1479,19.4076
2023-01-05,20.8501,0.1466,19.3868
2023-01-06,20.6357,0.1449,19.3186


In [83]:
positions = np.array([2300000, 23500000, 2500000])
resultados = var_es(df2, positions, 0.95, True)
resultados

,VAR %,VAR $,ES %,ES $
Resultados,-0.0108,"-992,484.2714",-0.0130,"-1,192,615.0249"


In [84]:
#                  Spot, Strike, r,     sigma, T
calls = np.array([ [462, 465, 0.0528, 0.17, 1/12],    #C1
                   [462, 470, 0.0528, 0.16, 1/12],    #C2
                   [462, 475, 0.0528, 0.15, 1/12],   #C3
                   [462, 465, 0.0523, 0.21, 3/12],    #C1
                   [462, 470, 0.0523, 0.2, 3/12],    #C2
                   [462, 475, 0.0523, 0.19, 3/12],
                   [462, 465, 0.0505, 0.22, 6/12],    #C1
                   [462, 470, 0.0505, 0.21, 6/12],    #C2
                   [462, 475, 0.0505, 0.2, 6/12]])  #C4

 #                  Spot, Strike, r,     sigma, T
puts =  np.array([ [462, 450, 0.0528, 0.24, 1/12],    #C1
                   [462, 445, 0.0528, 0.22, 1/12],    #C2
                   [462, 440, 0.0528, 0.2, 1/12],   #C3
                   [462, 450, 0.0523, 0.25, 3/12],    #C1
                   [462, 445, 0.0523, 0.24, 3/12],    #C2
                   [462, 440, 0.0523, 0.23, 3/12],
                   [462, 450, 0.0505, 0.26, 6/12],    #C1
                   [462, 445, 0.0505, 0.25, 6/12],    #C2
                   [462, 440, 0.0505, 0.24, 6/12]]) #P4

#                        C1, C2, C3, C4
noc_c = np.array([10,  12,  17,  18, 20, 12, 16, 14, 11])

#                        P1, P2, P3, P4
noc_p = np.array([26, 18, 14, 21, 26, 32, 17, 35, 43])

net_1 = delta_hedge(calls, puts, noc_c, noc_p)

if net_1 < 0:
    print(f'Hay que vender ${net_1:.4f} MDD del subyacente')
else:
    print(f'Hay que comprar ${net_1:.4f} MDD del subyacente')

Hay que vender $-6.9720 MDD del subyacente
